In [100]:
!pip install mysql

In [101]:
from sqlalchemy import create_engine
import mysql.connector
import pandas as pd
import os
import gzip
import spacy
import time

In [102]:
print("Loading language model...")
start = time.time()
nlp = spacy.load('en_core_web_md', entity=False)
print("...finished loading language model in {} seconds".format(time.time()-start))

Loading language model...
...finished loading language model in 13.43523097038269 seconds


In [103]:
def execute_query(query):
    """ Execute a query agains the local MySQL DB (note I allow login by root without a password)
    This function returns a list of results from the query, which may or may not be empty, depending on the query.
    """
    #cnx = mysql.connector.connect(user='root', password='rootroot',
    cnx = mysql.connector.connect(user='root',
                                  host='127.0.0.1', port='3306',
                                  #ssl_disabled = 'True',
                                  database='recommender',
                                  allow_local_infile = "True")
    cursor = cnx.cursor(buffered=True)
    cursor.execute(query)
    results = []
    for result in cursor:
        results.append(result)
    cnx.commit()
    cursor.close()
    cnx.close()
    
    return results

In [104]:
features_docs = []

def setup_features():
    """ Setup the spaCy doc instance for each feature. 
    This setup only needs to be run once, not for every time we evaluate a user query string."""
    
    results = execute_query('SELECT DISTINCT feature FROM consolidated_features')
    print("Found {} product features.".format(len(results)))
    features_items = []

    for result in results:
    
        # The following ugly code was required to strip stuff of the start and end of features that were in the DB 
        clean_result = result[0].strip('\", ')
        clean_result = clean_result.strip('\"')
        clean_result = clean_result.strip('\'')
    
        features_items.append(clean_result)
    
    for item in features_items:
        features_docs.append(nlp(item))
    
    features_items = None
    
    print("features like: ", features_docs[:5])
    
setup_features()

Found 774 product features.
features like:  [board, age group?my, well area, color pencil, small child]


In [105]:
test_string = 'toys games puzzles activities blue'

In [106]:
doc = nlp(test_string)

In [107]:
these_results = []
r_sims = []
for r_doc in features_docs:
    if (len(r_doc.text) > 0) and (r_doc.vector.any() > 0):
        similarity = r_doc.similarity(doc)
        r_sims.append((r_doc.text,similarity))
s_sims = sorted(r_sims, key=lambda pair: -pair[1])
r_sims = []

In [108]:
s_sims[:10]

[('toy fun', 0.7770468190171068),
 ('fun toy', 0.7770468190171068),
 ('moby!moby fun toy', 0.7770468097120159),
 ('fun game', 0.7676796069572991),
 ('game fun', 0.7676796069572991),
 ('many toy', 0.7399413691513244),
 ('good educational toy', 0.7365114352526856),
 ('great educational toy', 0.7341752190342793),
 ('nice educational toy', 0.730732335533269),
 ('educational toy', 0.724685982540135)]